### Objective

- To read in the data from multi-table excel worksheet
- Convert the tables to multiple pandas dataframes
- Convert the pandas Dataframe to Pyspark Dataframes
- Create pipeline in Pyspark for Regression Analysis
- Write out the pyspark dataframes to database 

In [ ]:
#Starting with import of pyspark and related modules

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark.pandas as ps
import warnings
warnings.filterwarnings("ignore")

from pyspark.ml import *

#lets get some excel files into the system
import pandas as pd
import openpyxl

In [2]:
#Initiating the spark session with postgres driver    
sparkSQL = SparkSession.builder.appName('Spark SQL') \
        .config('spark.jars',"/usr/share/java/postgresql-42.2.26.jar") \
        .getOrCreate()

22/11/30 06:33:26 WARN Utils: Your hostname, codeStation resolves to a loopback address: 127.0.1.1; using 172.17.0.1 instead (on interface docker0)
22/11/30 06:33:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/11/30 06:33:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/30 06:33:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
mllibPath = "mllib/"
externalData = "externalData/"
ytDE = "/home/solverbot/Desktop/ytDE/csvfiles"

In [4]:
sparkReader = sparkSQL.read

In [5]:
sparkContext = sparkSQL.sparkContext

In [6]:
sparkContext

<SparkContext master=local[*] appName=Spark SQL>

In [8]:
#I am trying to import a multi-worksheet XL, 
#the "Datasource" sheet is required sheet in the xl

datasource = pd.read_excel("sales Target Dashboard.xlsx",sheet_name="DataSource",
                          parse_dates=True)

In [9]:
datasource.columns

Index(['S/N', 'Date', 'Branch', 'Pizza Type', 'Quantity', 'Time', 'Time Range',
       'Price', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', 'Daily Target', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17', 'Sales Target', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Branch.1', 'Unnamed: 24',
       'Unnamed: 25'],
      dtype='object')

In [10]:
datasource.head()

,S/N,Date,Branch,Pizza Type,Quantity,Time,Time Range,Price,Unnamed: 8,Unnamed: 9,...,Unnamed: 16,Unnamed: 17,Sales Target,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Branch.1,Unnamed: 24,Unnamed: 25
0,1,42349,Abuja,Meatzaa,5,0.333345,Before 9:00am,8,NaN,NaN,...,NaN,NaN,Pizza,Sales Target,Revenue,NaN,NaN,Branch,Manager,Location
1,2,42352,Ibadan,Extravaganza,4,0.333356,Before 9:00am,8,NaN,NaN,...,NaN,NaN,BBQ Chicken,17280,NaN,NaN,NaN,Abuja,Christy Olson,Nigeria
2,3,42342,Ikoyi,BBQ Chicken,5,0.333380,Before 9:00am,16,NaN,NaN,...,NaN,NaN,BBQ Philly Steak,15232,NaN,NaN,NaN,Ibadan,Dan Peterson,Nigeria
3,4,42352,Ibadan,Extravaganza,1,0.333414,Before 9:00am,8,NaN,NaN,...,NaN,NaN,Beef Suya,11772,NaN,NaN,NaN,Ikoyi,Mable Lindsey,Nigeria
4,5,42345,Lekki,Meatzaa,4,0.333426,Before 9:00am,8,NaN,NaN,...,NaN,NaN,Chicken Bali,6400.8,NaN,NaN,NaN,Lekki,Kyle Carr,Nigeria


In [11]:
datasource.shape

(5000, 26)

In [12]:
columns = datasource.columns

In [13]:
columns

Index(['S/N', 'Date', 'Branch', 'Pizza Type', 'Quantity', 'Time', 'Time Range',
       'Price', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', 'Daily Target', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17', 'Sales Target', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Branch.1', 'Unnamed: 24',
       'Unnamed: 25'],
      dtype='object')

In [14]:
cleanedDS = datasource[['S/N', 'Date', 'Branch', 'Pizza Type', 'Quantity', 'Time', 'Time Range',
       'Price', 'Daily Target','Unnamed: 14','Sales Target', 'Unnamed: 19',
       'Branch.1', 'Unnamed: 24','Unnamed: 25']]

In [15]:
#Sale Data is important table
saleData = cleanedDS[['S/N', 'Date', 'Branch', 
                      'Pizza Type', 'Quantity', 'Time', 
                      'Time Range','Price']]
saleData.shape

(5000, 8)

In [16]:
saleData.columns

Index(['S/N', 'Date', 'Branch', 'Pizza Type', 'Quantity', 'Time', 'Time Range',
       'Price'],
      dtype='object')

In [17]:
#Intermediate table
miscellaneousData = cleanedDS[['Daily Target','Unnamed: 14',
                               'Sales Target', 'Unnamed: 19',
                               'Branch.1', 'Unnamed: 24',
                               'Unnamed: 25']]
miscellaneousData.shape

(5000, 7)

In [18]:
dailyTarget = miscellaneousData[["Daily Target","Unnamed: 14"]]

In [19]:
dailyTarget.columns = ["DailyTarget","SalesTarget"]

In [20]:
dailyTarget.dropna(axis=0,inplace=True)

In [21]:
dailyTarget = dailyTarget.iloc[1:,:]
dailyTarget.columns

Index(['DailyTarget', 'SalesTarget'], dtype='object')

In [22]:
branchTarget = miscellaneousData[["Branch.1","Unnamed: 24","Unnamed: 25"]]

In [23]:
branchTarget.columns = ["Branch","Manager","Location"]

In [24]:
branchTarget.dropna(axis=0,inplace=True)
branctTarget = branchTarget.iloc[1:,:]
branchTarget.columns

Index(['Branch', 'Manager', 'Location'], dtype='object')

In [25]:
productTarget = miscellaneousData[['Sales Target', 'Unnamed: 19']]
productTarget.columns = ['PizzaType','SalesTarget']
productTarget = productTarget.iloc[1:]
productTarget.dropna(axis=0,inplace=True)
productTarget

,PizzaType,SalesTarget
1,BBQ Chicken,17280
2,BBQ Philly Steak,15232
3,Beef Suya,11772
4,Chicken Bali,6400.8
5,Chicken Feast,5580.8
6,Chicken Legend,7064
7,Chicken Suya,13766.4
8,Extravaganza,5804.8
9,Hot Pepperoni Feast,17168
10,Hot Veggie,12160


### Pyarrow is required to convert pandas DF to pyspark DF
!pip install pyarrow

In [26]:
saleDataSparkDF = ps.from_pandas(saleData)
saleDataSparkDF

,S/N,Date,Branch,Pizza Type,Quantity,Time,Time Range,Price
0,1,42349,Abuja,Meatzaa,5,0.333345,Before 9:00am,8
1,2,42352,Ibadan,Extravaganza,4,0.333356,Before 9:00am,8
2,3,42342,Ikoyi,BBQ Chicken,5,0.333380,Before 9:00am,16
3,4,42352,Ibadan,Extravaganza,1,0.333414,Before 9:00am,8
4,5,42345,Lekki,Meatzaa,4,0.333426,Before 9:00am,8
5,6,42346,Surulere,Hot Veggie,2,0.333495,Before 9:00am,16
6,7,42342,Abuja,BBQ Philly Steak,5,0.333565,Before 9:00am,16
7,8,42345,Ibadan,Chicken Feast,1,0.333565,Before 9:00am,8
8,9,42345,Lekki,Meatzaa,3,0.333588,Before 9:00am,8
9,10,42342,Abuja,Chicken Suya,5,0.333623,Before 9:00am,16


In [33]:
psSaleDataDF = sparkSQL.createDataFrame(saleData)

In [34]:
psSaleDataDF.printSchema()

root
 |-- S/N: long (nullable = true)
 |-- Date: long (nullable = true)
 |-- Branch: string (nullable = true)
 |-- Pizza Type: string (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- Time: double (nullable = true)
 |-- Time Range: string (nullable = true)
 |-- Price: long (nullable = true)



In [ ]:
psSaleDataDF.write.csv()

In [32]:
#This is pyspark pandas variety of dataframe. 
#The regular Pyspark dataframe is sql variety. There are limits
type(saleDataSparkDF)

pyspark.pandas.frame.DataFrame

In [ ]:
help(saleDataSparkDF)

In [ ]:
#Helper functions to work with the database
def schemaGen(dataframe, schemaName):
    localSchema = pd.io.sql.get_schema(dataframe,schemaName)
    localSchema = localSchema.replace('TEXT','VARCHAR(255)').replace('INTEGER','NUMERIC').replace('\n','').replace('"',"")
    return "".join(localSchema)

#Using pandas read_sql for getting schema
def getSchema(tableName, credentials):
    schema = pd.read_sql("""SELECT table_catalog, table_name, 
                column_name, data_type, 
                ordinal_position, column_default, character_maximum_length,
                is_nullable FROM information_schema.columns where table_name='{}'""".format(tableName),con=credentials)
    return schema

#Issue is in using pd.read_sql to write data to the database. so using psycopg2
def queryTable(query):
    try:
        schema = cur.execute(query)
        return 
    except Exception as e:
        print(e)
        
#This doesn't return anything

#Using the pd.read_sql for getting data from db
def queryBase(query):
    requiredTable = pd.read_sql(query,con=credentials)
    return requiredTable

#This returns the dataframe

In [ ]:
schemaGen(saleData,'saleData')

### Pandas to Pyspark DF looses some power

Pyspark dataframe has sql related functions in-built into the object, while the pandas dataframe converted to pyspark df lacks this support.

We can fall back to the pandas own sql modules, and learn about the schema, and use the python database connection library like psycopg2 and query in the data.

Or, the Xlsx file can be converted to dataframe, then written out as csv files.Which then can be read into Pyspark context which has the database driver configured. 

There is mulitiple steps involved, so writing functions to do these two activities will save considerable effort. The same can be found in the next couple of cells

In [ ]:
saleData.to_csv("dashBoard_saleData.csv",index=False)
branctTarget.to_csv("dashBoard_branches.csv",index=False)
dailyTarget.to_csv("dashBoard_dailyTarget.csv",index=False)
productTarget.to_csv("dashBoard_productTarget.csv",index=False)

Need to escape the special characters, when using OS commands inside Jupyter Notebook

In [ ]:
%%sh
ls | grep dashBoard\*

Re-ingest the data into the pyspark environment, and then write into the postgres database. Idea here is to create another spark session inside a function, and then load the csv file, then move it into the database. 

Here goes nothing...

After experimenting a bit, I found it is better to initiate the session configured with the database jar files. Once the XL files are read, and cleaned using pandas, dataframe is written out into CSV file. 

The csv file path, the database name, database table name along with the spark session configured with the database driver, is used as parameters for the function, that writes to database. 

The same technique can be used for Spark dataframes also. In that case, the csv file will be replaced with spark dataframe itself. The database, tablename, and session will be still required. 


In [28]:
def writingCSVFiletoDatabase(session, csvFile,dbName,dbTableName):
    
    fileSparkDF = session.read.csv(csvFile,inferSchema=True,header=True)
    try:
        fileSparkDF.write \
                    .format('jdbc') \
                    .option("url", f"jdbc:postgresql://localhost:5432/{dbName}") \
                    .option('dbtable', dbTableName) \
                    .option('user','postgres') \
                    .option('password', 1234) \
                    .option('driver','org.postgresql.Driver') \
                    .save(mode='overwrite')
        print('Write Complete')
    except Exception as e:
        print(f'Write errored out due to {e}')
    

In [30]:
def writingSparkDFtoDatabase(sparkDF,dbName,dbTableName):
    
    try:
        sparkDF.write \
                    .format('jdbc') \
                    .option("url", f"jdbc:postgresql://localhost:5432/{dbName}") \
                    .option('dbtable', dbTableName) \
                    .option('user','postgres') \
                    .option('password', 1234) \
                    .option('driver','org.postgresql.Driver') \
                    .save(mode='overwrite')
        print('Write Complete')
    except Exception as e:
        print(f'Write errored out due to {e}')
    

In [ ]:
ritingCSVFiletoDatabase(sparkSQL,"dashBoard_saleData.csv","postgres","sale_data")

In [ ]:
saleDataSparkDF.

In [31]:
writingSparkDFtoDatabase(saleDataSparkDF,'postgres','trial_table')

Write errored out due to 'DataFrame' object has no attribute 'write'


#### Reading CSV files from multiple directories

Learning to partition the tables when the files are read from multiple folders is another important activity. Trying that with youtube data

In [ ]:
youtubeCSV = sparkReader.csv(ytDE,
                             recursiveFileLookup=True,
                             header=True,
                             inferSchema=True)

In [ ]:
youtubeCSV.count()

In [ ]:
youtubeCSV.printSchema()

In [ ]:
youtubeCSV.select("video_id", "trending_date", "category_id").show(3)

In [ ]:
#reading in malformed json
readJson = sparkContext.textFile("/home/solverbot/Desktop/ytDE/jsonfiles/CA_category_id.json")

In [ ]:
import json
with open("/home/solverbot/Desktop/ytDE/jsonfiles/CA_category_id.json") as js:
    data = js.read()
    print(type(data))
    jsonData = json.loads(data)
    jsonItems = jsonData['items']

In [ ]:
json.dump(obj=jsonItems,)

In [ ]:
with open("/home/solverbot/Desktop/ytDE/jsonfiles/CA_category.json","a") as jw:
    json.dump(jsonItems,jw)

In [ ]:
caJson = sparkReader.json("/home/solverbot/Desktop/ytDE/jsonfiles/CA_category.json")

In [ ]:
caJson.count()

In [ ]:
caJson.show(truncate=False)